In [15]:
pip install beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.


In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [51]:
# 네이버 주식 페이지 URL
stock_url = 'https://finance.naver.com/item/main.nhn?code=005380'  # 현대자동차 종목 코드

response = requests.get(stock_url)
soup = BeautifulSoup(response.content, 'html.parser')

In [52]:
# 재무 데이터가 있는 섹션 찾기
financial_section = soup.find('div', {'class': 'cop_analysis'})
financial_section1 = financial_section.find('div', {'class': 'sub_section'})
table = financial_section1.find('table', class_='tb_type1')

In [53]:
data = []
header_row = table.find('thead').find_all('th', scope='col')
header = [cell.get_text(strip=True) for cell in header_row if not any(item in cell.get_text() for item in ["IFRS연결", "최근 연간 실적", "최근 분기 실적"])]
data.append(header)

In [54]:
data

[['2020.12',
  '2021.12',
  '2022.12',
  '2023.12(E)',
  '2022.03',
  '2022.06',
  '2022.09',
  '2022.12',
  '2023.03',
  '2023.06(E)']]

In [55]:
data_table = table.find('tbody')
rows = data_table.find_all('tr')

for row in rows:
    row_data = [cell.get_text(strip=True) if cell.get_text(strip=True) != '' else '0' for cell in row.find_all(['td', 'th'])]
    data.append(row_data)

In [56]:
data[0].insert(0, '주요재무정보')

In [59]:
df = pd.DataFrame(data[1:], columns=data[0])
df = df.set_index('주요재무정보')
print(df)

             2020.12    2021.12    2022.12 2023.12(E)   2022.03   2022.06  \
주요재무정보                                                                      
매출액        1,039,976  1,176,106  1,425,275  1,598,305   302,986   359,999   
영업이익          23,947     66,789     98,198    145,970    19,289    29,798   
당기순이익         19,246     56,931     79,836    125,254    17,774    30,848   
영업이익률           2.30       5.68       6.89       9.13      6.37      8.28   
순이익률            1.85       4.84       5.60       7.84      5.87      8.57   
ROE(지배주주)       2.04       6.84       9.36      13.54      7.11      8.14   
부채비율          174.22     183.17     181.36          0    183.82    182.65   
당좌비율           44.87      45.68      51.97          0     49.35     48.01   
유보율         4,909.48   5,187.27   5,654.49          0  5,226.60  5,417.45   
EPS(원)         5,144     17,846     26,592     43,012     5,723    10,123   
PER(배)         37.33      11.71       5.68       4.33      9.61      8.01   

In [61]:
# 현재 날짜를 파일 이름으로 사용
current_date = datetime.now().strftime('%Y-%m-%d')

In [62]:
# 파일 경로 지정
file_path = f"/Users/leekyeongho/python/주식/{current_date}_financial_data.xlsx"

In [63]:
# 엑셀 파일로 저장
df.to_excel(file_path)
print(f"데이터프레임을 {file_path}에 저장했습니다.")

데이터프레임을 /Users/leekyeongho/python/주식/2023-08-22_financial_data.xlsx에 저장했습니다.
